# Notebook 10: Frequency-Domain Specifications and Loop Shaping
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores frequency-domain specifications and explores loop shaping for control synthesis, covering concepts like how to deal with disturbances/noise, how to approach control synthesis, and how to deal with unstable/non-minimum phase systems.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch


# Learning Objectives

After completing this material, you should be able to:

1. Understand and specify control requirements in the frequency domain:
   - Express command tracking requirements 
   - Specify disturbance rejection requirements
   - Define noise rejection specifications

2. Use loop shaping techniques for control synthesis:
   - Understand when to apply proportional, lead, and/or lag compensation.
   - Design proportional, lead, and lag compensators to achieve the control objective. 
   - Combine compensation elements systematically to derive the systems dynamic compensator.

3. Understand fundamental performance limitations:
   - Recognize the waterbed effect
   - Handle constraints from non-minimum phase zeros
   - Deal with limitations from unstable poles



## Required Packages



Run the following cell to import required packages:

In [ ]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider, HBox, VBox, Checkbox
from IPython.display import display, clear_output, HTML
from scipy import signal
import control

# Motivation

In the past few weeks we have introduced the frequency response, and corresponding tools (Bode, Nyquist) to assess closed-loop stability, stability margins and general system (frequency-conditioned) behavior. However, we had not yet seen how to (systematically) use the frequency response to achieve the control objective. Thus, this week we introduce frequency-domain specifications, the frequency equivalent of the time-domain specifications seen previously, along with how we can systematically design dynamic compensators -- through loop shaping --, to achieve the control objective.

# 1. Frequency-Domain Specifications


## 1.1 Introduction

So far we have considered (time) specifications in terms of output responses -- e.g., ensure that the system achieves $M\%$ overshoot with a rise time of $X$ seconds with no steady state error. However, control systems are typically exposed to disturbances and noise, which may correspondingly affect the ability to achieve such specifications. As an intuitive example, consider that we wish to regulate the temperature of a room. Suppose a dynamic compensator has been tuned such that it satisfies the time-domain specifications. However, if the temperature sensor exhibits a noisy temperature signal, it could result in the thermostat accounting for effects that are not present -- i.e., it does not need to change the temperature.

Disturbance and noise signals can be typically expressed in terms of some frequency band -- e.g., think that the noise from a sensor typically occurs at a high-frequency (greater than $100Hz$), whereas disturbances, such as the draft entering from below a door, typically occur at low frequencies (less than $10Hz$). Consequentially, we can express disturbance rejection and noise rejection by through distinct frequency bands:

- *Low frequencies*: At low frequencies, investigate command tracking and disturbance rejection.
- *High frequencies*: At high frequencies, investigate noise rejection.
- *Mid-frequencies*: In the middle of the frequency range, consider the stability margins and robustness.

Before introducing the frequency specifications, recall that for a closed-loop system with the below feedback architecture, we work with two key transfer functions:
<div style="text-align:center;">
<img src="./img/feedback_architecture.png" alt="Obstacle" width="500">
</div>

1. Sensitivity function: $S(s)= \frac{E(s)}{R(s)}= \frac{1}{1 + L(s)}$
   - $S(s)$ captures the system's ability to reject disturbances and determines error in tracking commands
2. Complementary sensitivity function: $T(s) = \frac{L(s)}{1 + L(s)}$
   - $T(s)$ captures the system's ability to track commands and determines the effect of noise.

Where $L(s) = C(s)P(s)$ is the loop transfer function.


## 1.2 Frequency Specifications


Frequency specifications are typically phrased in terms of achieving, good command tracking/disturbance rejection and noise rejection:

1. Command tracking/disturbance rejection:
   - $|S(j\omega)| \ll 1$ at low frequencies, 
   - Thus $|L(j\omega)|$ is large at low frequencies and hence $|T(j\omega)|\approx 1$ at low frequencies.
   - Example: *ensure commands are tracked with max 10% error up to 10Hz*

2. Noise rejection:
   - $|T(j\omega)| \ll 1$ at high frequencies  
   - Example: *ensure noise is reduced by factor of 10 at frequencies above 100Hz*

Since the closed-loop specifications are connected to the open-loop, given a certain specification -- *ensure $|L(s)|<a, \forall \omega < b Hz$ * --, it is possible to plot the corresponding "good region" in the open-loop bode plot. This, is introduced in the following section.

Furthermore, it is important to note that the above specifications can be quite generic in general and mixed and matched with time-domain specification as well as the phase and gain margins -- e.g., have a rise time of $X$ seconds whilst maintaining a phase margin of $Y^\circ$. Although for certain specifications it may be possible to derive expressions for specific systems (i.e., with what we did for time-domain specifications of $1^{\text{st}}$ and $2^{\text{nd}}$ order systems), approximations can be performed as they tend to hold (under certain assumptions), and the results can then be easily verified. Consequentially, if non-standard approximations are required, these will be provided in the question -- see the problems in the corresponding problem sheet for examples on this.


## 1.3 Translating to Open-Loop Specifications (Bode Obstacle Course)

The closed-loop specifications can be translated to requirements on the open-loop transfer function $L(s)$:

1. Command tracking/disturbance rejection requirement:
   - Recall that at low frequencies we want $|S(j\omega)| = |1+L(j\omega)|^{-1}$ to be small.
   - We can write this as $|S(j\omega)| \cdot |W_1(j\omega)| < 1$ 
   - This can be approximated as $|L(j\omega)| > |W_1(j\omega)|$

2. Noise rejection requirement:
   - $|T(j\omega)| \cdot |W_2(j\omega)| < 1$
   - This translates to $|L(j\omega)| < |W_2(j\omega)|^{-1}$

3. Cross-over frequency:
   - The frequency where $|L(j\omega_{gc})| = 1$ is called the (gain) crossover frequency $\omega_{gc}$

Where $W_1(j\omega)$ and $W_2(j\omega)$ are weighting functions that capture the desired behavior at low and high frequencies, respectively.

-  $W_1(j\omega)$ enforces performance at low frequencies (larger gains mean stricter tracking/disturbance rejection requirements)

-  $W_2(j\omega)$ enforces robustness at high frequencies (larger gains mean stricter noise attenuation requirements)

These create an "obstacle course" on the Bode plot through which $L(j\omega)$ must navigate to ensure that the frequency specifications are met. The goal would be to find a loop transfer function that stays between the two curves. 

<div style="text-align:center;">
<img src="./img/obstacle.png" alt="Obstacle" width="500">
</div>

The challenge is to design $L(j\omega)$ such that:

   - It stays above $W_1$​ at low frequencies
   - It stays below $W_2^{-1}$​ at high frequencies
   - It transitions smoothly between these bounds near crossover

# 2. Intro to Control Synthesis

Recall the time-domain specifications and how we used the root locus to tune controllers such that the specifications were met. Although, certain approaches were introduced, they mostly are based on trial and error -- e.g., pick some $k_p$, draw root locus, if locus does not go through the "good" region, add $k_d$, and try again. Further, since the "good" region is derived directly from the output response, as one changes the dynamic compensator, the "good" region varies. Frequency specifications provide us with tools for a more systematic approach to control synthesis -- i.e., to "steer" us through the Bode obstacle course. 

Below two approaches are introduced. The first is called the *brute force* whereby you design a transfer function that satisfies the requirements and then determine the appropriate controller. The second, is more systematic, and relies on our understanding of the different compensators to shape the Bode plot around the obstacles -- this method will be referred to as **Loop shaping**.


## 2.1 Brute Force




Suppose you have the open-loop system $L(s)=C(s)P(s)$, with some design specifications. You could:
1. Design a desired open-loop transfer function $L_{\text{des}}(s)$ that satisfies the requirements and has at least the same relative degree as $P(s)$ -- i.e., the same difference between number of poles and zeros. 
   - If $L_{\text{des}}$ has a lower relatively degree than $P(s)$, it would mean that $C(s)$ has a negative relative degree (i.e., more zeros than poles), which would make it a non-causal and physically unrealizable system.
2. Computing the controller $C(s) = \frac{L_{\text{des}}(s)}{P(s)}$ correspondingly.

However:
1. It may result in unnecessarily complicated controllers. 
2. It only works for open-loop stable and minimum-phase systems. 
   - Otherwise, the compensator with have unstable pole-zero cancellations.

## 2.2 Loop Shaping

The core idea behind loop shaping is to combine one or more of the elements (e.g., proportional gain, lead compensation, lag compensation) in series to design the dynamic compensator. It leverages that each element contributes uniquely to the frequency response, and thus we can directly "shape" the response to satisfy the specifications.

The lecture contains what each element is, how they are defined, and their effects. Thus, to avoid repetitive content, this is skipped here. Instead, we provide an interactive approach to visualizing the effects for each of the different elements. Below you will find a brief description for each of the components, and then a tool that lets you experiment with the fundamental building blocks of loop shaping.

1. **Proportional Gain (k)**:
   - Shifts the magnitude plot up/down uniformly
   - No effect on phase
   - Typically can increase the gain to improve command tracking (i.e., increase $|L(j\omega)$ at low frequencies) or increase the closed-loop bandwidth.
     - Potentially at the cost of stability.

2. **Lead Compensation** $\left(\frac{s/a + 1}{s/b + 1}, 0 < a < b\right)$:
   - Tool of choice for increasing phase margin. 
     - Maximum phase increase of $90^\circ$.
     - Maximum phase increase occurs around frequency $\sqrt{ab}$.
     - Max phase increase is equal to: $\varphi_{\text{max}} = 2 \arctan \left(\sqrt{\frac{b}{a}}\right) - 90^\circ$

   - Also:
     - Increases magnitude at high frequencies by $b/a$.
     - Magnitudes at low frequencies remains unaffected.
     - Increases the slope of the magnitude at frequencies between $a$ and $b$ by $+20 \text{dB}/\text{decade}$

3. **Lag Compensation** $\left(\frac{s/a + 1}{s/b + 1}, 0 < b < a\right)$:
   - Tool of choice for improving command tracking and disturbance rejection.
     - Decrease the phase around $\sqrt{ab}$ by up to $90^\circ$. 
     - Max phase decrease is equal to: $\varphi_{\text{min}} = 2 \arctan \left(\sqrt{\frac{b}{a}}\right) - 90^\circ$
     - Pick $a << \omega_c$, as to not affect the crossover frequency and phase margin too much.
   - Also:
     - Decreases the magnitude at high frequencies by $b/a$.
       - To decrease magnitude at low frequencies, pick $a/b$ as the desired increase in magnitude at low frequencies, and multiply $k$ by $a/b$ such that the magnitude at high frequencies remains unaffected.
     - Decreases the slope of the magnitude at frequencies between $b$ and $a$ by $20 \text{dB}/\text{decade}$.


### 2.2.1 Loop-Shaping Design Procedure


1. Start by plotting the uncompensated open-loop response (plant)
2. Identify key problems/specifications by comparing with bounds:
   - Low-frequency gain (for tracking/disturbance rejection)
   - Crossover frequency (for bandwidth/speed of response)
   - Phase margin (for stability/damping)
   - High-frequency roll-off (for noise rejection)

3. Apply compensation elements in this typical order:
   - Add gain K to roughly position the magnitude curve
   - Use lead compensation to improve phase margin around crossover
   - Use lag compensation if more low-frequency gain is needed
   - Fine-tune gain K if needed


To help intuitively visualize these behaviors and solve the problem, we have provided a tool below.
We have provided an obstacle course region (corresponding to the specifications above), and the plant's transfer function.

The plot shows:
- Plant response in green (try changing it with the numerator/denominator fields!)
- Compensated system in blue
- Red region: Must stay above this at low frequencies (> 50 or 34 dB for tracking spec)
- Yellow region: Must stay below this at high frequencies (noise rejection spec)



#### Interactive Exercise


1. Start with just proportional gain - what effects do you notice on the magnitude and phase?
2. Try adding lead compensation to improve phase margin. Where should you place $a$ and $b$?
3. If needed, add lag compensation for more low-frequency gain
4. Can you find a solution that meets all specifications? 

Play around with different combinations - what works? What doesn't? Can you find a simple solution that meets all specifications? There's often more than one way to succeed!

In [ ]:
def create_plant(num_str, den_str):
    """Create plant transfer function from string inputs"""
    try:
        num = [float(n) for n in num_str.split(',')]
        den = [float(d) for d in den_str.split(',')]
        return control.TransferFunction(num, den)
    except:
        # Return default plant if there's an error
        return control.TransferFunction([0.01, 0.2, 2], [1, 2, 2, 0])

def create_compensator(K, lead_a, lead_b, lag_a, lag_b, use_lead=True, use_lag=True):
    """Create compensator transfer function based on parameters
    For lead: (s/a + 1)/(s/b + 1) where 0 < a < b
    For lag: (s/a + 1)/(s/b + 1) where 0 < b < a"""
    comp = control.TransferFunction([K], [1])  # Start with gain
    
    if use_lead:
        # Lead compensator: (s/a + 1)/(s/b + 1) where 0 < a < b
        lead_num = [1/lead_a, 1]
        lead_den = [1/lead_b, 1]
        comp = comp * control.TransferFunction(lead_num, lead_den)
    
    if use_lag:
        # Lag compensator: (s/a + 1)/(s/b + 1) where 0 < b < a
        lag_num = [1/lag_a, 1]
        lag_den = [1/lag_b, 1]
        comp = comp * control.TransferFunction(lag_num, lag_den)
    
    return comp

def plot_bode(K=1, lead_a=0.3, lead_b=30, lag_a=10, lag_b=0.1, 
              use_lead=True, use_lag=True, show_bounds=True,
              num_str='0.01,0.2,2', den_str='1,2,2,0'):
    """Plot Bode diagram with compensator effects"""
    plt.figure()
    
    # Create plant and compensator
    plant = create_plant(num_str, den_str)
    comp = create_compensator(K, lead_a, lead_b, lag_a, lag_b, use_lead, use_lag)
    
    # Get loop transfer function
    L = plant * comp
    
    # Generate frequency points
    w = np.logspace(-2, 3, 1000)
    
    # Get frequency response
    mag, phase, w = control.bode(L, w, plot=False)
    
    # Plot magnitude
    plt.subplot(211)

    # Get and plot plant frequency response
    mag_p, phase_p, _ = control.bode(plant, w, plot=False)
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag_p), 'g-', label='Plant', alpha=0.7)

    plt.subplot(212)
    plt.semilogx(w, phase_p * 180/np.pi, 'g-', alpha=0.7)

    # Then plot L(s) in blue
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag), 'b-', label='Compensated system')
    
    # Add performance bounds
    if show_bounds:
        # W1 bound (low-frequency box for disturbance rejection)
        # Must stay ABOVE this at low frequencies
        w1_x = [w[0], 0.1, 0.1, w[0]]
        w1_y = [20, 20, -100, -100]
        plt.fill(w1_x, w1_y, alpha=0.5, color='r', label='Load disturbance\nattenuation')

        # W2 bound (high-frequency box for noise rejection)
        # Must stay BELOW this at high frequencies
        w2_x = [100, w[-1], w[-1], 100]
        w2_y = [100, 100, -25, -25]
        plt.fill(w2_x, w2_y, alpha=0.5, color='y', label='High frequency\nmeasurement noise')

    plt.grid(True)
    plt.ylabel('Magnitude [dB]')
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    
    # Add reference lines
    plt.axhline(y=0, color='k', linestyle=':')
    
    # Plot phase
    plt.subplot(212)
    plt.semilogx(w, phase * 180/np.pi, 'b-')
    plt.grid(True)
    plt.ylabel('Phase [deg]')
    plt.xlabel('Frequency [rad/s]')
    
    # Add phase margin indicators
    gain_crossover_idx = np.abs(mag - 1).argmin()
    if np.abs(mag[gain_crossover_idx] - 1) < 0.1:  # Only show if we actually cross 0dB
        gain_crossover = w[gain_crossover_idx]
        phase_at_crossover = phase[gain_crossover_idx] * 180/np.pi
        phase_margin = 180 + phase_at_crossover
        
        plt.subplot(211)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        
        plt.subplot(212)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        plt.plot(gain_crossover, phase_at_crossover, 'ro')
        plt.text(gain_crossover, phase_at_crossover, f'PM = {phase_margin:.1f}°')
        
        # Add reference lines
        plt.axhline(y=-180, color='k', linestyle=':')
    
    plt.tight_layout()

# Create interactive widgets
numerator_input = widgets.Text(
    value='0.01,0.2,2',
    description='Numerator:',
    style={'description_width': 'initial'}
)

denominator_input = widgets.Text(
    value='1,2,2,0',
    description='Denominator:',
    style={'description_width': 'initial'}
)

K_slider = FloatSlider(value=1, min=0.1, max=1000, step=0.1, description='Gain K')
lead_a_slider = FloatSlider(value=0.3, min=0.01, max=10, step=0.1, description='Lead a')
lead_b_slider = FloatSlider(value=30, min=1, max=100, step=1, description='Lead b')
lag_a_slider = FloatSlider(value=10, min=0.1, max=50, step=0.1, description='Lag a')
lag_b_slider = FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='Lag b')
use_lead = widgets.Checkbox(value=True, description='Use Lead')
use_lag = widgets.Checkbox(value=True, description='Use Lag')
show_bounds = widgets.Checkbox(value=True, description='Show Bounds')

# Create interactive plot
interactive_plot = interactive_output(plot_bode, 
                                   {'K': K_slider,
                                    'lead_a': lead_a_slider,
                                    'lead_b': lead_b_slider,
                                    'lag_a': lag_a_slider,
                                    'lag_b': lag_b_slider,
                                    'use_lead': use_lead,
                                    'use_lag': use_lag,
                                    'show_bounds': show_bounds,
                                    'num_str': numerator_input,
                                    'den_str': denominator_input})

# Display widgets and plot
display(VBox([
    HBox([numerator_input, denominator_input]),
    HBox([K_slider]),
    HBox([use_lead, lead_a_slider, lead_b_slider]),
    HBox([use_lag, lag_a_slider, lag_b_slider]),
    HBox([show_bounds]),
    interactive_plot
]))

### 2.2.2 Limitations: Unstable and non-minimum-phase systems


Previously, we have stated that, when relying on the Bode plot, in the case that the open-loop system has poles or zeros in the right-half plane, that all results should be checked (either with the Nyquist and/or root locus). Similarly, when performing loop shaping on an open-loop system with such poles/zeros, the same goes. However, it is possible to use the same loop shaping principles to tune a dynamic compensator if open-loop zeros/poles are present in the right-half plane -- but again, always double-check the results.

The main idea is to factor the plant transfer function into a "nicely behaved" plant $P_\text{mp}(s)$ and into a "bode demon" $D(s)$:
$$ P(s) = P_\text{mp}(s) D(s)$$

Where:
- $P_\text{mp}(s)$ is obtained by replacing all poles/zeros of P(s) in the r.h.p with their mirror image with respect to the imaginary axis. 
- $D(s)$ contains all the poles/zeros of $P(s)$ in the right half plane, times the inverse of $P_\text{mp}(s)$. Note that:
  - $|D(jw)| = 1, \forall \omega$ -- i.e., is an all-pass filter.
  - Choose sign of $D(s)$ so that the phase of $D(j\omega)$ is negative.
  - In essence, $D(s)$ alters the phase plot by introducing a phase lag, (and reducing the phase margin). It does not alter the magnitude plot.

#### Non-Minimum Phase Zeros

Consider for example a system with a non-minimum phase zero (a zero in the right half-plane). $D(s)$ has a phase lag of $\angle D(j\omega)= -2 \arctan(\frac{\omega}{z})$, and thus reduces the achievable phase margin by said factor. The practical consequence? We're forced to cross 0dB at lower frequencies, effectively slowing down our closed-loop system. The slower the zero (small z), the worse this limitation becomes.

#### Unstable Open-loop Systems

A similar story unfolds for unstable open-loop systems (those with poles in the right half-plane), but with a twist. Here the "Bode demon" adds a phase lag of $-2\arctan(\frac{p}{\omega})$. To stabilize such a system, we must cross 0dB at higher frequencies, requiring high bandwidth and gain. This translates to needing fast controllers and powerful actuators in practice. And just as slow non-minimum phase zeros are troublesome, fast unstable poles (large p) pose greater challenges.


#### Performance Limitations and the Bode Integral


These aren't just practical difficulties - they represent fundamental limitations in control design, captured in Bode's integral theorem:

> Let $S(s)$ be the sensitivity function of an internally stable closed-loop system with loop transfer function $L(s)$. Assume that $\lim_{s\to\infty}sL(s)=0$. Then:
>$$\int_0^{\infty} \log|S(j\omega)| d\omega = \pi\sum p_k$$

This formula tells us something profound: to reject disturbances in one frequency range (small sensitivity S), we must amplify them somewhere else. It's a mathematical expression of the "waterbed effect" - push down one part of the sensitivity function, and another part must rise. The presence of unstable poles makes this trade-off more severe.

This is what makes control design both challenging and fascinating. There's rarely a "perfect" solution - instead, we must carefully balance tradeoffs based on what matters most for our specific application. The interactive tool above lets you explore these tradeoffs in a simplified setting, but keeping these fundamental limitations in mind helps us set realistic expectations and make informed design choices in practice.

Let's visualize this waterbed effect directly. In the plot below, we have a simple feedback system with a PI controller. Try increasing the controller gain - as you make the sensitivity smaller at low frequencies (better disturbance rejection), watch how it inevitably increases at higher frequencies (worse noise rejection). This is the waterbed effect in action, and it's a fundamental limitation that no clever control design can circumvent!

Notice how the total area between the sensitivity curve and the 0dB line remains roughly constant - this is exactly what Bode's integral theorem tells us. When we have unstable poles, this area must become even larger, making our design tradeoffs even more challenging.

In [ ]:
def plot_waterbed_demo(gain=1):
    """
    Demonstration of the waterbed effect using real transfer functions
    """
    # Generate frequency points (linear scale)
    w = np.linspace(0.01, 100, 1000)

    # Create a simple plant
    # P(s) = 1/(s(s+1))
    plant = control.TransferFunction([1], [1, 1, 0])

    # Create controller with varying gain
    # C(s) = K - P controller
    controller = control.TransferFunction([gain], [1])

    # Calculate loop transfer function
    L = plant * controller

    # Calculate sensitivity
    S = control.feedback(1, L)

    # Get frequency response
    mag_S, phase_S, _ = control.bode(S, w, plot=False)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))

    # Sensitivity plot
    ax.plot(w, 20*np.log10(mag_S), 'b-', label='Sensitivity S')

    # Add 0dB line
    ax.axhline(y=0, color='k', linestyle=':')

    # Add shading to show area
    ax.fill_between(w, 0, 20*np.log10(mag_S), alpha=0.2, color='blue')

    # Calculate integral correctly
    # Using natural log for the integral
    log_S = np.log(abs(mag_S))  # natural log


    # The integral should be over all frequencies, but we approximate with our finite range
    w_estimate = np.logspace(-6,6,10000)
    mag_S_esitmate, phase_S, _ = control.bode(S, w_estimate, plot=False)
    log_S_esimate = np.log(abs(mag_S_esitmate))  # natural log

    integral = np.trapezoid(log_S_esimate, w_estimate)  # integrate
    integral = integral / np.pi  # normalize by π

    ax.set_title(f'Waterbed Effect (∫log|S(jω)|dlog(ω)/π ≈ {integral:.2f})')
    ax.grid(True)
    ax.set_ylabel('Magnitude [dB]')
    ax.set_xlabel('Frequency [rad/s]')
    ax.set_ylim([-40, 40])
    plt.tight_layout()

# Create slider for controller gain
gain_slider = FloatSlider(
    value=1,
    min=0.1, max=1000, step=0.1,
    description='Controller Gain'
)

# Create interactive plot
interactive_plot = interactive_output(plot_waterbed_demo,
    {'gain': gain_slider})

# Display widgets and plot
display(VBox([
    gain_slider,
    interactive_plot
]))